<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-001.png" class="img-responsive"/>
</div>



In [1]:
def count_chars(input_str):
    counts = {}

    for c in input_str:
        counts[c] = counts.get(c, 0) + 1

    return counts

print(count_chars('hello there'))

{'h': 2, 'e': 3, 'l': 2, 'o': 1, ' ': 1, 't': 1, 'r': 1}


<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-002.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-003.png" class="img-responsive"/>
</div>


In [2]:
def replace_pairs(lst, target_pair, combined_id):
    replaced_ids = []

    i = 0
    while i < len(lst):
        if tuple(lst[i:i+2]) == target_pair:
            replaced_ids.append(combined_id)
            i += 2
        else:
            replaced_ids.append(lst[i])
            i += 1

    return replaced_ids


print(replace_pairs([1, 2, 3, 1, 2], (1, 2), 4))

[4, 3, 4]


<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-004.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-005.png" class="img-responsive"/>
</div>


<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-006.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-007.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-008.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-009.png" class="img-responsive"/>
</div>


<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-010.png" class="img-responsive"/>
</div>




In [3]:
import os

try:
    import google.colab
    REPO_URL = "https://github.com/wtheisen/nd-cse-10124-lectures.git"

    REPO_NAME = "/content/nd-cse-10124-lectures"
    L_PATH = "nd-cse-10124-lectures/Datasets"

    %cd /content/
    !rm -r {REPO_NAME}

    # Clone repo
    if not os.path.exists(REPO_NAME):
        !git clone {REPO_URL}

        # cd into the data folder
        %cd {L_PATH}
        !pwd

except ImportError:
    print("Unable to download repo, either:")
    print("\tA.) You're not on colab")
    print("\tB.) It has already been cloned")


import utilities as uts

/content
rm: cannot remove '/content/nd-cse-10124-lectures': No such file or directory
Cloning into 'nd-cse-10124-lectures'...
remote: Enumerating objects: 203, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 203 (delta 122), reused 141 (delta 60), pack-reused 0 (from 0)
Receiving objects: 100% (203/203), 19.36 MiB | 26.30 MiB/s, done.
Resolving deltas: 100% (122/122), done.
/content/nd-cse-10124-lectures/Datasets
/content/nd-cse-10124-lectures/Datasets


In [4]:
print('Size of Jabberwocky Graph (Words):', len(uts.build_graph_word('jabberwocky.txt')))
print('Size of Zoomer Graph (Words):', len(uts.build_graph_word('zoomer.txt')))
print('Size of Shakespeare Graph (Words):', len(uts.build_graph_word('shakespeare.txt')))
print('#' * 80)
print('Size of Jabberwocky Graph (Chars):', len(uts.build_graph_char('jabberwocky.txt')))
print('Size of Shakespeare Graph (Chars):', len(uts.build_graph_char('shakespeare.txt')))
print('Size of Zoomer Graph (Chars):', len(uts.build_graph_char('zoomer.txt')))
print('#' * 80)

Size of Jabberwocky Graph (Words): 77
Size of Zoomer Graph (Words): 2378
Size of Shakespeare Graph (Words): 53164
################################################################################
Size of Jabberwocky Graph (Chars): 46
Size of Shakespeare Graph (Chars): 91
Size of Zoomer Graph (Chars): 90
################################################################################


<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-011.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-011.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-012.png" class="img-responsive"/>
</div>

In [5]:
print(list('hello there'.encode('utf-8')))

[104, 101, 108, 108, 111, 32, 116, 104, 101, 114, 101]


<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-013.png" class="img-responsive"/>
</div>

In [6]:
class Cat():
    def __init__(self, name, color):
        self.name = name
        self.color = color

cute_kitty = Cat('roger', 'orange')
print(type(cute_kitty))

<class '__main__.Cat'>


<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-014.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-015.png" class="img-responsive"/>
</div>

<div class="thumbnail">
    <img src="https://williamtheisen.com/nd-cse-10124-lectures/Lecture_Images/Lecture04/slide-016.png" class="img-responsive"/>
</div>

In [7]:
"""
Minimal (byte-level) Byte Pair Encoding tokenizer.

Algorithmically follows along the GPT tokenizer:
https://github.com/openai/gpt-2/blob/master/src/encoder.py
"""

class Simple_Tokenizer():
    def __init__(self):
        self.merges = {} # (int, int) -> int

        self.special_tokens = {'<|sos|>': 256, '<|eos|>': 257} # str -> int, e.g. {'<|endoftext|>': 100257}
        self.vocab = {idx: bytes([idx]) for idx in range(256)} | {idx: special.encode("utf-8") for special, idx in self.special_tokens.items()}

    def _count_pairs(self, ids, counts=None):
        """
        Given a list of integers, return a dictionary of counts of consecutive pairs
        Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
        Optionally allows to update an existing dictionary of counts
        """
        counts = {} if counts is None else counts

        for pair in zip(ids, ids[1:]): # iterate consecutive elements
            counts[pair] = counts.get(pair, 0) + 1

        return counts

    def _merge_pairs(self, ids, pair, idx):
        """
        In the list of integers (ids), replace all consecutive occurrences
        of pair with the new integer token idx
        Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
        """
        newids = []
        i = 0

        while i < len(ids):
            # if not at the very last position AND the pair matches, replace it
            if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
                newids.append(idx)
                i += 2
            else:
                newids.append(ids[i])
                i += 1

        return newids

    def train(self, text, max_vocab_size):
        vocab_size = len(self.vocab)
        num_merges = max_vocab_size - vocab_size

        # input text preprocessing
        text_bytes = text.encode("utf-8") # raw bytes
        ids = list(text_bytes) # list of integers in range 0..255

        for i in range(num_merges):
            # count up the number of times every consecutive pair appears
            stats = self._count_pairs(ids)

            # find the pair with the highest count
            pair = max(stats, key=stats.get)

            # mint a new token: assign it the next available id
            idx = 256 + i

            # replace all occurrences of pair in ids with idx
            ids = self._merge_pairs(ids, pair, idx)

            # save the merge
            self.merges[pair] = idx
            self.vocab[idx] = self.vocab[pair[0]] + self.vocab[pair[1]]

    def decode(self, ids):
        # given ids (list of integers), return Python string
        text_bytes = b"".join(self.vocab[idx] for idx in ids)
        text = text_bytes.decode("utf-8", errors="replace")
        return text

    def encode(self, text):
        import re
        special_pattern = "(" + "|".join(re.escape(k) for k in self.special_tokens) + ")"
        special_chunks = re.split(special_pattern, text)

        encoded_str = []
        for chunk in special_chunks[1:-1]:
            if chunk in self.special_tokens:
                # this is a special token, encode it separately as a special case
                encoded_str.append(self.special_tokens[chunk])
            else:
                # given a string text, return the token ids
                text_bytes = chunk.encode("utf-8") # raw bytes
                chunk_ids = list(text_bytes) # list of integers in range 0..255

                while len(chunk_ids) >= 2:
                    # find the pair with the lowest merge index
                    counted_pairs = self._count_pairs(chunk_ids)
                    earliest_pair = min(counted_pairs, key=lambda p: self.merges.get(p, float("inf")))

                    # just the first pair in the list, arbitrarily
                    # we can detect this terminating case by a membership check
                    if earliest_pair not in self.merges:
                        break # nothing else can be merged anymore

                    # otherwise let's merge the best pair (lowest merge index)
                    pair_idx = self.merges[earliest_pair]
                    chunk_ids = self._merge_pairs(chunk_ids, earliest_pair, pair_idx)

                encoded_str += chunk_ids

        return encoded_str

    def visualize_tokenization(self, ids):
        """Small helper function useful in debugging: visualize the tokenization of render_conversation"""
        GREEN = '\033[92m'
        RESET = '\033[0m'
        GRAY = '\033[90m'

        tokens = []
        for token_id in ids:
            token_str = self.decode([token_id])
            tokens.append(f"{GREEN}{token_str}{GRAY}({token_id}){RESET}")

            if token_str in ['<|sos|>', '<|eos|>']:
                tokens.append('\n\n\t')

        return ' | '.join(tokens)

In [8]:
#print('Size of Jabberwocky Graph (Tokens):', len(uts.create_token_graph('jabberwocky.txt')))
#print('Size of Shakespeare Graph (Tokens):', len(uts.create_token_graph('shakespeare.txt')))
# Size of Shakespeare Graph (Tokens): 286 (this takes like ~7 minutes to run)

num_gens = 3

char_graph = uts.build_graph_char('zoomer.txt')
print('Character Level Generation [zoomer.txt]:')
for i in range(0, num_gens):
  print(f'\t{i}: {''.join(uts.generate_sequence(char_graph, 50))}')

print('#' * 80)

#for vocab_size in [258, 512, 1024, 2048, 4096]:
for vocab_size in [512]:
    s_t = Simple_Tokenizer()

    token_graph = uts.build_graph_token('zoomer.txt', s_t, vocab_size=vocab_size, special_tokens=True)
    print('Size of Zoomer Graph (Tokens, No Regex):', len(token_graph))

    print(f'Token Level Generation (No Regex, {vocab_size} Vocab Size) [zoomer.txt]:')
    for i in range(0, num_gens):
        print(f'\t{i}: {''.join(uts.generate_sequence(token_graph, 50))}')
    print('#' * 80)

print('#' * 80)

word_graph = uts.build_graph_word('zoomer.txt')
print('Word Level Generation [zoomer.txt]:')
for i in range(0, num_gens):
  print(f'\t{i}: {' '.join(uts.generate_sequence(word_graph, 50))}')


Character Level Generation [zoomer.txt]:
	0: <|sos|>M.<|eos|>
	1: <|sos|>I’st ang, viome wabunirisou’rke sorlaroupan athee 
	2: <|sos|>The?<|eos|>
################################################################################
Size of Zoomer Graph (Tokens, No Regex): 279
Token Level Generation (No Regex, 512 Vocab Size) [zoomer.txt]:
	0: <|sos|><|eos|>
	1: <|sos|><|eos|>
	2: <|sos|><|eos|>
################################################################################
################################################################################
Word Level Generation [zoomer.txt]:
	0: <|sos|> <|eos|>
	1: <|sos|> <|eos|>
	2: <|sos|> <|eos|>


In [11]:
from tokenizer import RegexTokenizer

r_t = RegexTokenizer()

print(r_t.GPT4_SPLIT_PATTERN)

print(r_t.chunk("No cap are u rolling the party tonight?"))

for vocab_size in [512, 1024, 2048, 4096]:
    r_t = RegexTokenizer()

    token_graph = uts.build_graph_token('zoomer.txt', r_t, special_tokens=True, vocab_size=vocab_size)
    print('#' * 80)
    print(f'''Size of Zoomer Graph (Tokens, Regex, {vocab_size} Vocab Size):', len(token_graph))

Token Level Generation (Regex, {vocab_size} Vocab Size) [zoomer.txt]:''')

    for i in range(0, num_gens):
        print(f'\t{i}: {''.join(uts.generate_sequence(token_graph, 50))}')

'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+
['No', ' cap', ' are', ' u', ' rolling', ' the', ' party', ' tonight', '?']
################################################################################
Size of Zoomer Graph (Tokens, Regex, 512 Vocab Size):', len(token_graph))

Token Level Generation (Regex, 512 Vocab Size) [zoomer.txt]:
	0: <|sos|>ilsending meme thing offlo miner, guq!<|eos|>
	1: <|sos|>tg with bac.<|eos|>
	2: <|sos|>i5 mhiuks get’s late for appening.<|eos|>


KeyboardInterrupt: 

## Export to HTML

Uncomment the final line of the cell below and run it to export this notebook to HTML

In [ ]:
import os, json

def export_notebook():
  L_PATH = "nd-cse-10124-lectures/Notebooks"
  L = "Lecture_04_Tokenization"

  try:
      from google.colab import _message, files

      # where you WANT it to live (repo folder)
      repo_ipynb_path = f"/content/{L_PATH}/{L}.ipynb"

      # grab current notebook contents from the UI
      nb = _message.blocking_request("get_ipynb", timeout_sec=1)["ipynb"]

      # write it into the repo folder as a real file
      os.makedirs(os.path.dirname(repo_ipynb_path), exist_ok=True)
      with open(repo_ipynb_path, "w", encoding="utf-8") as f:
          json.dump(nb, f)

      # convert + download pdf
      !jupyter nbconvert --to html "{repo_ipynb_path}"
      files.download(repo_ipynb_path.replace(".ipynb", ".html"))
  except:
      import subprocess

      nb_fp = os.getcwd() + f'{L}.ipynb'
      print(os.getcwd())

      subprocess.run(["jupyter", "nbconvert", "--to", "html", nb_fp], check=True)

#export_notebook()